<a href="https://colab.research.google.com/github/Rajatvs/ML-2/blob/main/Bayesian.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
# =====================================================
# BAYESIAN NETWORK (pgmpy >= 1.0 compliant version)
# Predicting likelihood of an event using Bayesian Net
# Google Colab Ready - Single Code Block
# =====================================================

!pip install pgmpy pandas numpy

import pandas as pd
import numpy as np
from pgmpy.models import DiscreteBayesianNetwork
from pgmpy.estimators import MaximumLikelihoodEstimator
from pgmpy.inference import VariableElimination

# -----------------------------------------------------
# 1. CREATE SAMPLE DATA
# -----------------------------------------------------
np.random.seed(42)

data_size = 500

data = pd.DataFrame({
    "Weather": np.random.choice(["Sunny", "Rainy"], size=data_size, p=[0.7, 0.3]),
    "Speeding": np.random.choice(["Yes", "No"], size=data_size, p=[0.4, 0.6]),
    "RoadCondition": np.random.choice(["Good", "Bad"], size=data_size, p=[0.8, 0.2]),
})

def accident_prob(row):
    base = 0.05
    if row["Weather"] == "Rainy":
        base += 0.15
    if row["Speeding"] == "Yes":
        base += 0.20
    if row["RoadCondition"] == "Bad":
        base += 0.25
    return "Yes" if np.random.rand() < base else "No"

data["Accident"] = data.apply(accident_prob, axis=1)

print("Sample Data:")
print(data.head())

# -----------------------------------------------------
# 2. BUILD BAYESIAN NETWORK STRUCTURE (new API)
# -----------------------------------------------------
model = DiscreteBayesianNetwork([
    ("Weather", "Accident"),
    ("Speeding", "Accident"),
    ("RoadCondition", "Accident")
])

# -----------------------------------------------------
# 3. LEARN PARAMETERS USING MLE
# -----------------------------------------------------
model.fit(data, estimator=MaximumLikelihoodEstimator)

# -----------------------------------------------------
# 4. INFERENCE ENGINE
# -----------------------------------------------------
infer = VariableElimination(model)

# QUERY 1: Accident probability given rainy + speeding
q1 = infer.query(
    variables=["Accident"],
    evidence={"Weather": "Rainy", "Speeding": "Yes"}
)
print("\nLikelihood of Accident (Rainy, Speeding=Yes):")
print(q1)

# QUERY 2: Accident probability given perfect conditions
q2 = infer.query(
    variables=["Accident"],
    evidence={"Weather": "Sunny", "Speeding": "No", "RoadCondition": "Good"}
)
print("\nLikelihood of Accident (Sunny, No Speeding, Good Road):")
print(q2)

# -----------------------------------------------------
# 5. PRINT LEARNED CPDs
# -----------------------------------------------------
print("\n===== LEARNED CPDs (Conditional Probability Tables) =====")
for cpd in model.get_cpds():
    print(cpd)
    print()





Sample Data:
  Weather Speeding RoadCondition Accident
0   Sunny       No          Good       No
1   Rainy       No          Good       No
2   Rainy      Yes           Bad      Yes
3   Sunny       No          Good       No
4   Sunny       No           Bad       No

Likelihood of Accident (Rainy, Speeding=Yes):
+---------------+-----------------+
| Accident      |   phi(Accident) |
+===============+=================+
| Accident(No)  |          0.5777 |
+---------------+-----------------+
| Accident(Yes) |          0.4223 |
+---------------+-----------------+

Likelihood of Accident (Sunny, No Speeding, Good Road):
+---------------+-----------------+
| Accident      |   phi(Accident) |
+===============+=================+
| Accident(No)  |          0.9739 |
+---------------+-----------------+
| Accident(Yes) |          0.0261 |
+---------------+-----------------+

===== LEARNED CPDs (Conditional Probability Tables) =====
+----------------+-------+
| Weather(Rainy) | 0.308 |
+-------------